05 — Create Analytical Views & Derived Metrics
================================================
Lumina Forecasting Hub

These BigQuery views sit on top of the fact tables and power the Looker dashboard.
They pre-compute key metrics so Looker stays fast and the logic lives in SQL, not LookML.

Views Created:
  v_grid_operations_hourly    — Enriched hourly demand with BA metadata + forecast accuracy
  v_grid_operations_daily     — Daily aggregates for trend analysis
  v_forecast_accuracy_summary — MAPE, RMSE, bias by BA and time period
  v_generation_fuel_mix       — Monthly gen with renewable flags + shares
  v_energy_transition_tracker — YoY renewable share change by state
  v_retail_cost_analysis      — Retail metrics with derived cost indicators
  v_renewable_price_scatter   — Renewable share vs electricity price by state (Page 4)
  v_executive_kpis            — Top-level KPIs for scorecard cards

Usage in Colab:
  1. Run notebooks 01-04 first (schema + all ingestion)
  2. Set GCP_PROJECT_ID below
  3. Run all cells — views are CREATE OR REPLACE so re-running is safe

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery

GCP_PROJECT_ID = "YOUR_GCP_PROJECT_ID"   # <-- UPDATE
BQ_DATASET     = "lumina"

client = bigquery.Client(project=GCP_PROJECT_ID)
FQ = f"{GCP_PROJECT_ID}.{BQ_DATASET}"    # Fully-qualified prefix

def run_ddl(sql, name):
    """Execute DDL and print status."""
    client.query(sql).result()
    print(f"  ✓ {name}")

In [ ]:
print("=== Page 1: Grid Operations ===")

# ── v_grid_operations_hourly ─────────────────────────────────────────
run_ddl(f"""
CREATE OR REPLACE VIEW `{FQ}.v_grid_operations_hourly` AS
SELECT
    h.timestamp_utc,
    DATE(h.timestamp_utc) AS date_utc,
    EXTRACT(HOUR FROM h.timestamp_utc) AS hour_utc,
    EXTRACT(DAYOFWEEK FROM h.timestamp_utc) AS day_of_week,
    CASE WHEN EXTRACT(DAYOFWEEK FROM h.timestamp_utc) IN (1, 7) THEN TRUE ELSE FALSE END AS is_weekend,

    h.ba_code,
    ba.ba_name,
    ba.region,
    ba.peak_capacity_mw,

    h.demand_mw,
    h.demand_forecast_mw,
    h.net_generation_mw,
    h.interchange_mw,
    h.forecast_error_mw,
    h.forecast_error_pct,

    -- Derived metrics
    SAFE_DIVIDE(h.demand_mw, ba.peak_capacity_mw) * 100 AS capacity_utilization_pct,
    ABS(h.forecast_error_pct) AS abs_forecast_error_pct,
    CASE
        WHEN SAFE_DIVIDE(h.demand_mw, ba.peak_capacity_mw) > 0.90 THEN 'CRITICAL'
        WHEN SAFE_DIVIDE(h.demand_mw, ba.peak_capacity_mw) > 0.80 THEN 'WARNING'
        WHEN SAFE_DIVIDE(h.demand_mw, ba.peak_capacity_mw) > 0.70 THEN 'ELEVATED'
        ELSE 'NORMAL'
    END AS capacity_alert_level,

    -- Net position: positive = exporting, negative = importing
    CASE
        WHEN h.interchange_mw > 0 THEN 'Exporting'
        WHEN h.interchange_mw < 0 THEN 'Importing'
        ELSE 'Balanced'
    END AS trade_position,

FROM `{FQ}.fact_hourly_demand` h
LEFT JOIN `{FQ}.dim_balancing_authority` ba ON h.ba_code = ba.ba_code
""", "v_grid_operations_hourly")


# ── v_grid_operations_daily ──────────────────────────────────────────
run_ddl(f"""
CREATE OR REPLACE VIEW `{FQ}.v_grid_operations_daily` AS
SELECT
    DATE(timestamp_utc) AS date_utc,
    ba_code,
    ba_name,
    region,
    peak_capacity_mw,

    -- Daily demand stats
    ROUND(AVG(demand_mw), 1) AS avg_demand_mw,
    ROUND(MAX(demand_mw), 1) AS peak_demand_mw,
    ROUND(MIN(demand_mw), 1) AS min_demand_mw,
    ROUND(MAX(demand_mw) - MIN(demand_mw), 1) AS demand_range_mw,

    -- Daily forecast accuracy
    ROUND(AVG(ABS(forecast_error_pct)), 2) AS mape_daily,
    ROUND(SQRT(AVG(POWER(forecast_error_mw, 2))), 1) AS rmse_daily,
    ROUND(AVG(forecast_error_mw), 1) AS bias_mw,

    -- Capacity utilization
    ROUND(MAX(SAFE_DIVIDE(demand_mw, peak_capacity_mw)) * 100, 1) AS peak_utilization_pct,
    COUNTIF(SAFE_DIVIDE(demand_mw, peak_capacity_mw) > 0.90) AS hours_above_90pct,

    -- Generation & trade
    ROUND(AVG(net_generation_mw), 1) AS avg_generation_mw,
    ROUND(AVG(interchange_mw), 1) AS avg_interchange_mw,

    COUNT(*) AS hours_reported

FROM `{FQ}.v_grid_operations_hourly`
GROUP BY 1, 2, 3, 4, 5
""", "v_grid_operations_daily")


# ── v_forecast_accuracy_summary ──────────────────────────────────────
run_ddl(f"""
CREATE OR REPLACE VIEW `{FQ}.v_forecast_accuracy_summary` AS
WITH hourly AS (
    SELECT
        ba_code,
        ba_name,
        DATE_TRUNC(DATE(timestamp_utc), MONTH) AS month,
        demand_mw,
        demand_forecast_mw,
        forecast_error_mw,
        forecast_error_pct,
        hour_utc,
        is_weekend
    FROM `{FQ}.v_grid_operations_hourly`
    WHERE demand_mw IS NOT NULL AND demand_forecast_mw IS NOT NULL
)
SELECT
    ba_code,
    ba_name,
    month,

    -- Overall accuracy
    ROUND(AVG(ABS(forecast_error_pct)), 2) AS mape,
    ROUND(SQRT(AVG(POWER(forecast_error_mw, 2))), 1) AS rmse_mw,
    ROUND(AVG(forecast_error_mw), 1) AS mean_bias_mw,
    ROUND(STDDEV(forecast_error_mw), 1) AS std_error_mw,

    -- Peak vs off-peak accuracy
    ROUND(AVG(CASE WHEN hour_utc BETWEEN 7 AND 22 THEN ABS(forecast_error_pct) END), 2) AS mape_peak_hours,
    ROUND(AVG(CASE WHEN hour_utc NOT BETWEEN 7 AND 22 THEN ABS(forecast_error_pct) END), 2) AS mape_offpeak_hours,

    -- Weekday vs weekend
    ROUND(AVG(CASE WHEN NOT is_weekend THEN ABS(forecast_error_pct) END), 2) AS mape_weekday,
    ROUND(AVG(CASE WHEN is_weekend THEN ABS(forecast_error_pct) END), 2) AS mape_weekend,

    COUNT(*) AS total_observations

FROM hourly
GROUP BY 1, 2, 3
""", "v_forecast_accuracy_summary")

print()

In [ ]:
print("=== Page 2: Energy Mix & Transition ===")

# ── v_generation_fuel_mix ────────────────────────────────────────
run_ddl(f"""
CREATE OR REPLACE VIEW `{FQ}.v_generation_fuel_mix` AS
SELECT
    CAST(g.period_month AS DATE) AS period_month,
    EXTRACT(YEAR FROM g.period_month) AS year,
    EXTRACT(MONTH FROM g.period_month) AS month,

    g.state_code,
    geo.state_name,
    geo.census_region,
    geo.census_division,

    g.fuel_code,
    f.fuel_label,
    f.is_renewable,
    f.emission_factor_kg_mwh,

    CAST(g.sector_code AS STRING) AS sector_code,
    g.generation_mwh,

    -- Estimated CO2 from generation (using emission factors)
    ROUND(g.generation_mwh * COALESCE(f.emission_factor_kg_mwh, 0) / 1e6, 2) AS estimated_co2_ktons,

FROM `{FQ}.fact_monthly_generation` g
LEFT JOIN `{FQ}.dim_fuel_type` f ON g.fuel_code = f.fuel_code
LEFT JOIN `{FQ}.dim_geography` geo ON g.state_code = geo.state_code
""", "v_generation_fuel_mix")


# ── v_energy_transition_tracker ──────────────────────────────────
# Deduplicate by (month, state, fuel) to avoid double-counting
# across sector codes, then pivot fuel types into columns.
run_ddl(f"""
CREATE OR REPLACE VIEW `{FQ}.v_energy_transition_tracker` AS
WITH deduped AS (
    SELECT *,
        ROW_NUMBER() OVER (
            PARTITION BY period_month, state_code, fuel_code
            ORDER BY generation_mwh DESC NULLS LAST
        ) AS rn
    FROM `{FQ}.v_generation_fuel_mix`
),
monthly_totals AS (
    SELECT
        period_month,
        state_code,
        state_name,
        census_region,
        SUM(generation_mwh) AS total_gen_mwh,
        SUM(CASE WHEN is_renewable THEN generation_mwh ELSE 0 END) AS renewable_gen_mwh,
        SUM(CASE WHEN fuel_label = 'Coal' THEN generation_mwh ELSE 0 END) AS coal_gen_mwh,
        SUM(CASE WHEN fuel_label = 'Natural Gas' THEN generation_mwh ELSE 0 END) AS gas_gen_mwh,
        SUM(CASE WHEN fuel_label IN ('Solar', 'Solar PV', 'Solar Thermal', 'Distributed PV') THEN generation_mwh ELSE 0 END) AS solar_gen_mwh,
        SUM(CASE WHEN fuel_label = 'Wind' THEN generation_mwh ELSE 0 END) AS wind_gen_mwh,
        SUM(CASE WHEN fuel_label = 'Nuclear' THEN generation_mwh ELSE 0 END) AS nuclear_gen_mwh,
        SUM(estimated_co2_ktons) AS estimated_co2_ktons
    FROM deduped
    WHERE rn = 1
    GROUP BY 1, 2, 3, 4
)
SELECT
    *,
    ROUND(SAFE_DIVIDE(renewable_gen_mwh, total_gen_mwh) * 100, 2) AS renewable_share_pct,
    ROUND(SAFE_DIVIDE(coal_gen_mwh, total_gen_mwh) * 100, 2) AS coal_share_pct,
    ROUND(SAFE_DIVIDE(gas_gen_mwh, total_gen_mwh) * 100, 2) AS gas_share_pct,
    ROUND(SAFE_DIVIDE(solar_gen_mwh, total_gen_mwh) * 100, 2) AS solar_share_pct,
    ROUND(SAFE_DIVIDE(wind_gen_mwh, total_gen_mwh) * 100, 2) AS wind_share_pct,
    ROUND(SAFE_DIVIDE(nuclear_gen_mwh, total_gen_mwh) * 100, 2) AS nuclear_share_pct,
    ROUND(SAFE_DIVIDE(estimated_co2_ktons * 1e3, total_gen_mwh / 1e3) * 1e3, 1) AS grid_carbon_intensity_kg_mwh,
    LAG(SAFE_DIVIDE(renewable_gen_mwh, total_gen_mwh) * 100, 12) OVER (
        PARTITION BY state_code ORDER BY period_month
    ) AS renewable_share_pct_prev_year,
FROM monthly_totals
""", "v_energy_transition_tracker")

print()


In [ ]:
print("=== Page 4: Cost & Market Intelligence ===")

# ── v_retail_cost_analysis ───────────────────────────────────────────
run_ddl(f"""
CREATE OR REPLACE VIEW `{FQ}.v_retail_cost_analysis` AS
SELECT
    CAST(r.period_month AS DATE) AS period_month,
    EXTRACT(YEAR FROM r.period_month) AS year,

    r.state_code,
    geo.state_name,
    geo.census_region,
    geo.population,

    CAST(r.sector_code AS STRING) AS sector_code,
    CASE CAST(r.sector_code AS STRING)
        WHEN 'RES' THEN 'Residential'
        WHEN 'COM' THEN 'Commercial'
        WHEN 'IND' THEN 'Industrial'
        WHEN 'TRA' THEN 'Transportation'
        WHEN 'OTH' THEN 'Other'
        ELSE CAST(r.sector_code AS STRING)
    END AS sector_name,

    r.revenue_musd,
    r.sales_mwh,
    r.price_cents_kwh,
    r.customers,

    -- Derived
    ROUND(SAFE_DIVIDE(r.revenue_musd * 1e6, r.customers), 2) AS revenue_per_customer_usd,
    ROUND(SAFE_DIVIDE(r.sales_mwh * 1e3, r.customers), 2) AS kwh_per_customer,

    -- Price ranking within period
    RANK() OVER (
        PARTITION BY r.period_month, r.sector_code
        ORDER BY r.price_cents_kwh DESC
    ) AS price_rank_high_to_low,

FROM `{FQ}.fact_retail_sales` r
LEFT JOIN `{FQ}.dim_geography` geo ON r.state_code = geo.state_code
""", "v_retail_cost_analysis")


# ── v_renewable_price_scatter ────────────────────────────────────────
# Joins renewable share (from generation data) with residential
# electricity price (from retail sales) by state for the latest year.
# Powers the Page 4 scatter plot: "Do high-renewable states pay more?"
run_ddl(f"""
CREATE OR REPLACE VIEW `{FQ}.v_renewable_price_scatter` AS
WITH latest_year AS (
    SELECT MAX(EXTRACT(YEAR FROM period_month)) AS yr
    FROM `{FQ}.v_energy_transition_tracker`
    WHERE total_gen_mwh > 0
),
renewable AS (
    SELECT
        state_code,
        ROUND(AVG(renewable_share_pct), 2) AS avg_renewable_share,
        ROUND(SUM(total_gen_mwh), 0) AS total_gen_mwh,
        ROUND(AVG(coal_share_pct), 2) AS avg_coal_share,
        ROUND(AVG(gas_share_pct), 2) AS avg_gas_share,
        ROUND(AVG(wind_share_pct), 2) AS avg_wind_share,
        ROUND(AVG(solar_share_pct), 2) AS avg_solar_share,
        ROUND(AVG(nuclear_share_pct), 2) AS avg_nuclear_share
    FROM `{FQ}.v_energy_transition_tracker`
    WHERE EXTRACT(YEAR FROM period_month) = (SELECT yr FROM latest_year)
    GROUP BY state_code
),
price AS (
    SELECT
        state_code,
        ROUND(AVG(price_cents_kwh), 2) AS avg_residential_price,
        ROUND(SUM(revenue_musd), 2) AS total_revenue_musd,
        ROUND(SUM(sales_mwh), 0) AS total_sales_mwh
    FROM `{FQ}.v_retail_cost_analysis`
    WHERE sector_name = 'Residential'
        AND year = (SELECT yr FROM latest_year)
    GROUP BY state_code
)
SELECT
    r.state_code,
    geo.state_name,
    geo.census_region,
    geo.population,
    r.avg_renewable_share,
    r.total_gen_mwh,
    r.avg_coal_share,
    r.avg_gas_share,
    r.avg_wind_share,
    r.avg_solar_share,
    r.avg_nuclear_share,
    p.avg_residential_price,
    p.total_revenue_musd,
    p.total_sales_mwh,

    -- Quadrant classification (16 ¢/kWh = current national avg)
    CASE
        WHEN r.avg_renewable_share > 25 AND p.avg_residential_price <= 16 THEN 'High Renewable, Low Cost'
        WHEN r.avg_renewable_share > 25 AND p.avg_residential_price > 16 THEN 'High Renewable, High Cost'
        WHEN r.avg_renewable_share <= 25 AND p.avg_residential_price <= 16 THEN 'Low Renewable, Low Cost'
        ELSE 'Low Renewable, High Cost'
    END AS quadrant

FROM renewable r
LEFT JOIN price p ON r.state_code = p.state_code
LEFT JOIN `{FQ}.dim_geography` geo ON r.state_code = geo.state_code
WHERE r.avg_renewable_share IS NOT NULL
    AND p.avg_residential_price IS NOT NULL
""", "v_renewable_price_scatter")

print()

In [ ]:
print("=== Executive KPIs ===")

run_ddl(f"""
CREATE OR REPLACE VIEW `{FQ}.v_executive_kpis` AS
WITH demand_kpis AS (
    SELECT
        ROUND(AVG(demand_mw), 0) AS current_avg_demand_mw,
        ROUND(MAX(demand_mw), 0) AS peak_demand_mw,
        ROUND(AVG(ABS(forecast_error_pct)), 2) AS overall_mape,
        ROUND(SQRT(AVG(POWER(forecast_error_mw, 2))), 0) AS overall_rmse_mw,
        COUNT(DISTINCT ba_code) AS active_bas,
        MIN(timestamp_utc) AS data_start,
        MAX(timestamp_utc) AS data_end
    FROM `{FQ}.fact_hourly_demand`
    WHERE timestamp_utc >= TIMESTAMP_SUB(
        (SELECT MAX(timestamp_utc) FROM `{FQ}.fact_hourly_demand`),
        INTERVAL 30 DAY
    )
),
gen_kpis AS (
    SELECT
        ROUND(SUM(CASE WHEN is_renewable THEN generation_mwh ELSE 0 END) /
              NULLIF(SUM(generation_mwh), 0) * 100, 1) AS national_renewable_share_pct,
        ROUND(SUM(generation_mwh) / 1e6, 1) AS total_generation_twh
    FROM `{FQ}.v_generation_fuel_mix`
    WHERE period_month >= DATE_SUB(
        CAST((SELECT MAX(period_month) FROM `{FQ}.fact_monthly_generation`) AS DATE),
        INTERVAL 12 MONTH
    )
),
retail_kpis AS (
    SELECT
        ROUND(AVG(price_cents_kwh), 2) AS national_avg_price_cents,
        ROUND(SUM(revenue_musd) / 1e3, 1) AS total_revenue_busd
    FROM `{FQ}.fact_retail_sales`
    WHERE CAST(sector_code AS STRING) = 'RES'
        AND period_month >= DATE_SUB(
            CAST((SELECT MAX(period_month) FROM `{FQ}.fact_retail_sales`) AS DATE),
            INTERVAL 12 MONTH
        )
)
SELECT
    d.current_avg_demand_mw,
    d.peak_demand_mw,
    d.overall_mape AS forecast_mape_pct,
    d.overall_rmse_mw AS forecast_rmse_mw,
    d.active_bas,
    g.national_renewable_share_pct,
    g.total_generation_twh,
    r.national_avg_price_cents,
    r.total_revenue_busd,
    d.data_start,
    d.data_end
FROM demand_kpis d
CROSS JOIN gen_kpis g
CROSS JOIN retail_kpis r
""", "v_executive_kpis")


In [ ]:
print("\n=== All Views in Dataset ===")
verify_query = f"""
SELECT table_name, table_type
FROM `{FQ}.INFORMATION_SCHEMA.TABLES`
WHERE table_type = 'VIEW'
ORDER BY table_name
"""
df_views = client.query(verify_query).to_dataframe()
print(df_views.to_string(index=False))
print(f"\nTotal views: {len(df_views)}")
print("\nAnalytical layer complete! These views are ready to connect to Looker.")